In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import add, activations
from keras.layers import Conv2D, MaxPooling2D, Dropout
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

C:\Users\Lokesh Vairamuthu\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset = pd.read_csv("HPDemand.csv")
dataset.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,27/7/2012,100.0
1,Product_0979,Whse_J,Category_028,19/1/2012,500.0
2,Product_0979,Whse_J,Category_028,3/2/2012,500.0
3,Product_0979,Whse_J,Category_028,9/2/2012,500.0
4,Product_0979,Whse_J,Category_028,2/3/2012,500.0


In [19]:
dataset_sam = dataset.sample(frac=0.4)
dataset_sam['Code'] = dataset_sam['Product_Code'].str.split('_').str[1]
dataset_sam['Category'] = dataset_sam['Product_Category'].str.split('_').str[1]
dataset_sam['Year'] = dataset_sam['Date'].str.split('/').str[2]
dataset_sam['Month'] = dataset_sam['Date'].str.split('/').str[1]
dataset_sam['Day'] = dataset_sam['Date'].str.split('/').str[0]
dataset_sam['PLC'] = dataset_sam['Warehouse'].str.split('_').str[1]
dataset_sam = dataset_sam.reset_index().drop(columns={"index","Product_Category","Product_Code","Date", "Warehouse"})

In [20]:
dataset_sam["PLC"] = [ord(x)%32 for x in dataset_sam["PLC"]]
dataset_sam = dataset_sam[['Code', 'Category', 'Year', 'Month', 'Day', 'PLC', 'Order_Demand']]
dataset_sam = dataset_sam.dropna().reset_index().drop(columns={"index"})

In [21]:
dataset_2 = dataset_sam
dataset_sam.head()

,Code,Category,Year,Month,Day,PLC,Order_Demand
0,0409,019,2012,11,20,10,5.0
1,0657,011,2014,4,25,10,2.0
2,1853,001,2012,6,1,10,1.0
3,1672,026,2014,1,14,10,1.0
4,1419,019,2015,2,24,10,400.0


In [22]:
hpd = np.array(dataset_sam)
cd = hpd[:,0]
cd = cd.astype(int)
hpd[:,0] = cd

cat = hpd[:,1]
cat = cat.astype(int)
hpd[:,1] = cat

yr = hpd[:,2]
yr = yr.astype(int)
hpd[:,2] = yr

mn = hpd[:,3]
mn = mn.astype(int)
hpd[:,3] = mn

dy = hpd[:,4]
dy = dy.astype(int)
hpd[:,4] = dy

In [23]:
from sklearn.preprocessing import MinMaxScaler

X = hpd[:,0:6]
y = hpd[:,6]
y = np.reshape(y, (-1,1))
scaler = MinMaxScaler()
print(scaler.fit(X))
print(scaler.fit(y))
xscale=scaler.transform(X)
yscale=scaler.transform(y)

X_train, X_test, y_train, y_test = train_test_split(xscale, yscale, test_size=0.33, random_state=42)

C:\Users\Lokesh Vairamuthu\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Lokesh Vairamuthu\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


MinMaxScaler(copy=True, feature_range=(0, 1))
MinMaxScaler(copy=True, feature_range=(0, 1))


In [24]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((268750, 6), (132370, 6), (268750, 1), (132370, 1))

In [25]:
model = Sequential()
model.add(Dense(16, input_dim=6, activation='relu', kernel_initializer='normal'))
model.add(Dense(4, input_dim=6, activation='relu'))
model.add(Dense(1, activation='linear'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                112       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 68        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 185
Trainable params: 185
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])

In [27]:
history = model.fit(X_train, y_train, epochs=150, batch_size=50,  verbose=1, validation_split=0.2)

Train on 215000 samples, validate on 53750 samples
Epoch 1/150
215000/215000 [==============================] - 6s 29us/step - loss: 4.9778e-05 - mean_squared_error: 4.9778e-05 - mean_absolute_error: 0.0019 - val_loss: 6.3703e-05 - val_mean_squared_error: 6.3703e-05 - val_mean_absolute_error: 0.0019
Epoch 2/150
215000/215000 [==============================] - 5s 25us/step - loss: 4.9739e-05 - mean_squared_error: 4.9739e-05 - mean_absolute_error: 0.0019 - val_loss: 6.3857e-05 - val_mean_squared_error: 6.3857e-05 - val_mean_absolute_error: 0.0016
Epoch 3/150
215000/215000 [==============================] - 5s 25us/step - loss: 4.9735e-05 - mean_squared_error: 4.9735e-05 - mean_absolute_error: 0.0019 - val_loss: 6.3797e-05 - val_mean_squared_error: 6.3797e-05 - val_mean_absolute_error: 0.0017
Epoch 4/150
215000/215000 [==============================] - 5s 25us/step - loss: 4.9762e-05 - mean_squared_error: 4.9762e-05 - mean_absolute_error: 0.0019 - val_loss: 6.3982e-05 - val_mean_squared_e

Epoch 66/150
215000/215000 [==============================] - 5s 24us/step - loss: 4.9769e-05 - mean_squared_error: 4.9769e-05 - mean_absolute_error: 0.0019 - val_loss: 6.4227e-05 - val_mean_squared_error: 6.4227e-05 - val_mean_absolute_error: 0.0024
Epoch 67/150
215000/215000 [==============================] - 5s 24us/step - loss: 4.9757e-05 - mean_squared_error: 4.9757e-05 - mean_absolute_error: 0.0019 - val_loss: 6.3809e-05 - val_mean_squared_error: 6.3809e-05 - val_mean_absolute_error: 0.0021
Epoch 68/150
215000/215000 [==============================] - 5s 24us/step - loss: 4.9807e-05 - mean_squared_error: 4.9807e-05 - mean_absolute_error: 0.0019 - val_loss: 6.3766e-05 - val_mean_squared_error: 6.3766e-05 - val_mean_absolute_error: 0.0021
Epoch 69/150
215000/215000 [==============================] - 5s 25us/step - loss: 4.9768e-05 - mean_squared_error: 4.9768e-05 - mean_absolute_error: 0.0019 - val_loss: 6.3863e-05 - val_mean_squared_error: 6.3863e-05 - val_mean_absolute_error: 0.0

Epoch 99/150
215000/215000 [==============================] - 5s 24us/step - loss: 4.9733e-05 - mean_squared_error: 4.9733e-05 - mean_absolute_error: 0.0019 - val_loss: 6.3951e-05 - val_mean_squared_error: 6.3951e-05 - val_mean_absolute_error: 0.0016
Epoch 100/150
215000/215000 [==============================] - 5s 24us/step - loss: 4.9792e-05 - mean_squared_error: 4.9792e-05 - mean_absolute_error: 0.0019 - val_loss: 6.3711e-05 - val_mean_squared_error: 6.3711e-05 - val_mean_absolute_error: 0.0018
Epoch 101/150
215000/215000 [==============================] - 6s 27us/step - loss: 4.9762e-05 - mean_squared_error: 4.9762e-05 - mean_absolute_error: 0.0019 - val_loss: 6.4220e-05 - val_mean_squared_error: 6.4220e-05 - val_mean_absolute_error: 0.0014
Epoch 102/150
215000/215000 [==============================] - 5s 24us/step - loss: 4.9754e-05 - mean_squared_error: 4.9754e-05 - mean_absolute_error: 0.0019 - val_loss: 6.3781e-05 - val_mean_squared_error: 6.3781e-05 - val_mean_absolute_error: 

In [28]:
ynew=model.predict([X_test])
print("Accuracy of the model", np.mean(np.abs(ynew-y_test)))

Accuracy of the model 0.0021860273991416886


In [29]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X_train, y_train)
lr.score(X_train, y_train)
y_pred = lr.predict(X_test)
print("Accuaracy of Linear Regression Model", np.mean(np.abs(y_pred - y_test)))

Accuaracy of Linear Regression Model 0.0018408309326222836


# Improving the accuracy of the prediction model by 25x

In [30]:
dataset_2 = dataset_2.drop(columns={"Category", "PLC"})

In [31]:
dataset_2.head()

,Code,Year,Month,Day,Order_Demand
0,0409,2012,11,20,5.0
1,0657,2014,4,25,2.0
2,1853,2012,6,1,1.0
3,1672,2014,1,14,1.0
4,1419,2015,2,24,400.0


In [32]:
hpd1 = np.array(dataset_sam)
cda = hpd1[:,0]
cda = cda.astype(int)
hpd[:,0] = cda

yra = hpd1[:,2]
yra = yra.astype(int)
hpd1[:,2] = yra

mna = hpd1[:,3]
mna = mna.astype(int)
hpd1[:,3] = mna

dya = hpd1[:,4]
dya = dya.astype(int)
hpd1[:,4] = dya

In [33]:
from sklearn.preprocessing import MinMaxScaler

X1 = hpd[:,0:4]
y1 = hpd[:,4]
y1 = np.reshape(y1, (-1,1))
scaler = MinMaxScaler()
print(scaler.fit(X1))
print(scaler.fit(y1))
xscale1=scaler.transform(X1)
yscale1=scaler.transform(y1)

X_train1, X_test1, y_train1, y_test1 = train_test_split(xscale1, yscale1, test_size=0.33, random_state=42)

MinMaxScaler(copy=True, feature_range=(0, 1))
MinMaxScaler(copy=True, feature_range=(0, 1))


C:\Users\Lokesh Vairamuthu\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Lokesh Vairamuthu\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [34]:
X_train1.shape, X_test1.shape, y_train1.shape, y_test1.shape

((268750, 4), (132370, 4), (268750, 1), (132370, 1))

In [35]:
model1 = Sequential()
model1.add(Dense(16, input_dim=4, activation='relu', kernel_initializer='normal'))
model1.add(Dense(4, input_dim=4, activation='relu'))
model1.add(Dense(1, activation='linear'))
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 16)                80        
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 68        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 5         
Total params: 153
Trainable params: 153
Non-trainable params: 0
_________________________________________________________________


In [36]:
model1.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])

In [37]:
history1 = model1.fit(X_train1, y_train1, epochs=150, batch_size=50,  verbose=1, validation_split=0.2)

Train on 215000 samples, validate on 53750 samples
Epoch 1/150
215000/215000 [==============================] - 6s 26us/step - loss: 0.0927 - mean_squared_error: 0.0927 - mean_absolute_error: 0.2587 - val_loss: 0.0853 - val_mean_squared_error: 0.0853 - val_mean_absolute_error: 0.2528
Epoch 2/150
215000/215000 [==============================] - 5s 25us/step - loss: 0.0870 - mean_squared_error: 0.0870 - mean_absolute_error: 0.2552 - val_loss: 0.0849 - val_mean_squared_error: 0.0849 - val_mean_absolute_error: 0.2524
Epoch 3/150
215000/215000 [==============================] - 5s 25us/step - loss: 0.0864 - mean_squared_error: 0.0864 - mean_absolute_error: 0.2546 - val_loss: 0.0856 - val_mean_squared_error: 0.0856 - val_mean_absolute_error: 0.2531
Epoch 4/150
215000/215000 [==============================] - 5s 25us/step - loss: 0.0861 - mean_squared_error: 0.0861 - mean_absolute_error: 0.2542 - val_loss: 0.0855 - val_mean_squared_error: 0.0855 - val_mean_absolute_error: 0.2529
Epoch 5/150
2

Epoch 36/150
215000/215000 [==============================] - 5s 24us/step - loss: 0.0857 - mean_squared_error: 0.0857 - mean_absolute_error: 0.2538 - val_loss: 0.0848 - val_mean_squared_error: 0.0848 - val_mean_absolute_error: 0.2522
Epoch 37/150
215000/215000 [==============================] - 5s 25us/step - loss: 0.0857 - mean_squared_error: 0.0857 - mean_absolute_error: 0.2538 - val_loss: 0.0848 - val_mean_squared_error: 0.0848 - val_mean_absolute_error: 0.2523
Epoch 38/150
215000/215000 [==============================] - 5s 24us/step - loss: 0.0857 - mean_squared_error: 0.0857 - mean_absolute_error: 0.2538 - val_loss: 0.0849 - val_mean_squared_error: 0.0849 - val_mean_absolute_error: 0.2523
Epoch 39/150
215000/215000 [==============================] - 5s 24us/step - loss: 0.0857 - mean_squared_error: 0.0857 - mean_absolute_error: 0.2538 - val_loss: 0.0850 - val_mean_squared_error: 0.0850 - val_mean_absolute_error: 0.2524
Epoch 40/150
215000/215000 [==============================] 

Epoch 71/150
215000/215000 [==============================] - 5s 24us/step - loss: 0.0858 - mean_squared_error: 0.0858 - mean_absolute_error: 0.2541 - val_loss: 0.0850 - val_mean_squared_error: 0.0850 - val_mean_absolute_error: 0.2526
Epoch 72/150
215000/215000 [==============================] - 5s 24us/step - loss: 0.0858 - mean_squared_error: 0.0858 - mean_absolute_error: 0.2541 - val_loss: 0.0850 - val_mean_squared_error: 0.0850 - val_mean_absolute_error: 0.2526
Epoch 73/150
215000/215000 [==============================] - 5s 24us/step - loss: 0.0858 - mean_squared_error: 0.0858 - mean_absolute_error: 0.2541 - val_loss: 0.0851 - val_mean_squared_error: 0.0851 - val_mean_absolute_error: 0.2526
Epoch 74/150
215000/215000 [==============================] - 5s 25us/step - loss: 0.0858 - mean_squared_error: 0.0858 - mean_absolute_error: 0.2541 - val_loss: 0.0851 - val_mean_squared_error: 0.0851 - val_mean_absolute_error: 0.2526
Epoch 75/150
215000/215000 [==============================] 

Epoch 106/150
215000/215000 [==============================] - 5s 24us/step - loss: 0.0858 - mean_squared_error: 0.0858 - mean_absolute_error: 0.2541 - val_loss: 0.0851 - val_mean_squared_error: 0.0851 - val_mean_absolute_error: 0.2526
Epoch 107/150
215000/215000 [==============================] - 6s 26us/step - loss: 0.0858 - mean_squared_error: 0.0858 - mean_absolute_error: 0.2541 - val_loss: 0.0851 - val_mean_squared_error: 0.0851 - val_mean_absolute_error: 0.2526
Epoch 108/150
215000/215000 [==============================] - 5s 25us/step - loss: 0.0858 - mean_squared_error: 0.0858 - mean_absolute_error: 0.2541 - val_loss: 0.0851 - val_mean_squared_error: 0.0851 - val_mean_absolute_error: 0.2526
Epoch 109/150
215000/215000 [==============================] - 5s 24us/step - loss: 0.0858 - mean_squared_error: 0.0858 - mean_absolute_error: 0.2541 - val_loss: 0.0851 - val_mean_squared_error: 0.0851 - val_mean_absolute_error: 0.2526
Epoch 110/150
215000/215000 [===========================

Epoch 141/150
215000/215000 [==============================] - 5s 24us/step - loss: 0.0858 - mean_squared_error: 0.0858 - mean_absolute_error: 0.2541 - val_loss: 0.0850 - val_mean_squared_error: 0.0850 - val_mean_absolute_error: 0.2526
Epoch 142/150
215000/215000 [==============================] - 5s 24us/step - loss: 0.0858 - mean_squared_error: 0.0858 - mean_absolute_error: 0.2541 - val_loss: 0.0850 - val_mean_squared_error: 0.0850 - val_mean_absolute_error: 0.2526
Epoch 143/150
215000/215000 [==============================] - 5s 24us/step - loss: 0.0858 - mean_squared_error: 0.0858 - mean_absolute_error: 0.2541 - val_loss: 0.0850 - val_mean_squared_error: 0.0850 - val_mean_absolute_error: 0.2526
Epoch 144/150
215000/215000 [==============================] - 5s 24us/step - loss: 0.0858 - mean_squared_error: 0.0858 - mean_absolute_error: 0.2541 - val_loss: 0.0850 - val_mean_squared_error: 0.0850 - val_mean_absolute_error: 0.2526
Epoch 145/150
215000/215000 [===========================

In [38]:
ynew1=model.predict([X_test])
print("Accuracy of the model", np.mean(np.abs(ynew1-y_test1)))

Accuracy of the model 0.482246903623002


In [39]:
from sklearn.linear_model import LinearRegression

lr1 = LinearRegression().fit(X_train1, y_train1)
lr1.score(X_train1, y_train1)
y_pred1 = lr1.predict(X_test1)
print("Accuaracy of Linear Regression Model", np.mean(np.abs(y_pred1 - y_test1)))

Accuaracy of Linear Regression Model 0.25360296730889426
